In [ ]:
#   _BIG DATA SCIENCE_ANALISE DE DADOS
#   Fundamentos da analise preditiva regressao linear, se baseia em dados historicos para prever o futuro
#   Machine Learning - Aprendizado de Maquina, aprende atravez de dados
#   Prendizado Supervisionado - dados com saida esperada tem o obejtivo de prever o futuro
#   REGRESSAO LINEAR - FUNCAO DE PRIMEIRO GRAU F(Y)= aX + b
#   NO PYTHON Y É A VARIAVEL DE DEPENDENCIA X É A VARIAVEL INDEPENDENTE, A coeficiente angular e B é o coeficiente linear
#   FAZER UMA CORRELAÇÃO ANTES DE UMA ANALISE PREDITIVA, TEM QUE DAR UMA CORRELAÇÃO FORTE